In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 原理
## 加载数据集

In [ ]:
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.3, random_state=42)
plt.scatter(X[y == 0, 0], X[y == 0, 1])
plt.scatter(X[y == 1, 0], X[y == 1, 1])
plt.show()

## 处理数据集

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 划分
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=666)
# 标准化
std_scaler = StandardScaler()
std_scaler.fit(X_train, y_train)
X_train = std_scaler.transform(X_train)
X_test = std_scaler.transform(X_test)


def fit(model):
    model.fit(X_train, y_train)
    print(model, ':', model.score(X_test, y_test))
    return model

## 训练模型

In [ ]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

svm_clf = fit(SVC())
log_clf = fit(LogisticRegression())
dt_clf  = fit(DecisionTreeClassifier(random_state=666))

## 投票预测

In [ ]:
y_predict1 = svm_clf.predict(X_test)
y_predict2 = log_clf.predict(X_test)
y_predict3 = dt_clf.predict(X_test)

In [ ]:
y_predict = np.array((y_predict1 + y_predict2 + y_predict3) >= 2, dtype='int')  # 投票: 少数服从多数

In [ ]:
""" 分类问题评价指标:
(1) 准确度： accuracy_score
(2) 精确度： precision_score   <--- 下面3个指标，用于偏斜数据集
(3) 召回率： recall_score
(4) F1值 ：  f1_score
"""
from sklearn.metrics import accuracy_score

print('支持向量机:', accuracy_score(y_test, y_predict1))
print('逻辑回归  :', accuracy_score(y_test, y_predict2))
print('决策树    :', accuracy_score(y_test, y_predict3))
print('集成学习  :', accuracy_score(y_test, y_predict))  # 准确度提高了

# Voting
- 单模型，每个算法关注的都是同一份数据，然后对结果进行投票表决

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
def VotingClassifierTest(voting):
    def VotingClassifierPrivate(svm_clf, log_clf, dt_clf, voting):
        # 1.创建集成学习分类器
        voting_clf = VotingClassifier(estimators=[
            ('支持向量机:', svm_clf),
            ('逻辑回归:', log_clf),
            ('决策树:', dt_clf)
        ], voting=voting)

        # 2.训练
        voting_clf.fit(X_train, y_train)

        # 3.预测
        print(voting_clf.score(X_test, y_test))

    if voting == 'soft':
        svm_clf = SVC(probability=True)  # 先计算概率
        VotingClassifierPrivate(svm_clf, log_clf, dt_clf, voting='soft')
    else:
        svm_clf = SVC(probability=False)
        VotingClassifierPrivate(svm_clf, log_clf, dt_clf, voting='hard')

## Hard Voting
- **投票:** 少数服从多数

In [ ]:
VotingClassifierTest(voting='hard')

## Soft Voting
- **投票:** 引入权重，要求`模型能计算概率`

In [ ]:
VotingClassifierTest(voting='soft')

# bagging
- **原理:** 创建更多的子模型，让子模型彼此具有差异性，子模型不需要很高的准确率，最终模型整体的准确率却是很高的
  - 子模型差异性: 
    - 只关注一部分数据: max_samples
    - 只关注一部分特征: max_features
    - 只关注一部分数据+特征: max_samples, max_features
  - 取样方式:
    - 放回取样: bootstrap=True, bootstrap_features=True
    - 不放回取样: bootstrap=False, bootstrap_features=False
  - oob(out of bag)
    - 放回取样，导致一部分样本很有可能没有取到。可以使用这部分没有取到的样本做测试/验证。设置: oob_score=True
  - 并行化处理
    - 每个模型都是独立的，可以并行。设置: n_jobs=?

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

**(1) 只关注一部分数据**

In [ ]:
bagging_clf = BaggingClassifier(DecisionTreeClassifier(),
                                n_estimators=500, # 创建 500 个子模型
                                max_samples=100,  # 每个子模型只关注 100 个数据
                                bootstrap=True,   # True: 放回取样
                                oob_score=True)   # True: 放回取样时，使用未取到的数据点进行预测

bagging_clf.fit(X, y)
bagging_clf.oob_score_

**(2) 只关注一部分特征**

In [ ]:
bagging_subspaces_clf = BaggingClassifier(DecisionTreeClassifier(),
                                          n_estimators=500,          # 创建 500 个子模型
                                          max_features=1,            # 每个子模型只关注 1 个特征
                                          bootstrap_features=True,   # True: 放回取样
                                          oob_score=True)            # True: 放回取样时，使用未取到的数据进行预测

bagging_subspaces_clf.fit(X, y)
bagging_subspaces_clf.oob_score_

**(3) 只关注一部分数据+特征**

In [ ]:
bagging_patches_clf = BaggingClassifier(DecisionTreeClassifier(),
                                        n_estimators=500,          # 创建 500 个子模型
                                        # 1.random samples
                                        max_samples=100,           # 每个子模型只关注 100 个数据
                                        bootstrap=True,            # True: 放回取样
                                        # 2.random features
                                        max_features=1,            # 每个子模型只关注 1 个特征
                                        bootstrap_features=True,   # True: 放回取样
                                        # 3.out of bag
                                        oob_score=True)            # True: 放回取样时，使用未取到的数据进行预测

bagging_patches_clf.fit(X, y)
bagging_patches_clf.oob_score_

## 随机森林
- 属于集成学习
- 集成了bagging和决策树
- 每个子模型都是决策树，只关注一部分数据or特征

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=500,  # 创建 500 个子模型
                                oob_score=True,    # True: 放回取样时，使用未取到的数据进行预测
                                random_state=666,
                                n_jobs=-1)
print(rf_clf.fit(X, y))
print(rf_clf.oob_score_)

## Extra-Trees
- 与随机森林不同的是：每棵决策树在节点划分上，采用完全随机的维度和阈值，而随机森林会搜索最优的维度和阈值

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
ExtraTreesClassifier()

In [ ]:
et_clf = ExtraTreesClassifier(n_estimators=500,  # 创建 500 个子模型
                              oob_score=True,    # True: 放回取样时，使用未取到的数据进行预测
                              bootstrap=True,    # True: 放回取样
                              random_state=666,
                              n_jobs=-1)
print(et_clf.fit(X, y))
print(et_clf.oob_score_)

# 集成学习解决回归问题

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
bagging_reg = BaggingRegressor(DecisionTreeRegressor(), n_estimators=500, oob_score=True)
bagging_reg.fit(X, y)
bagging_reg.oob_score_

In [ ]:
rf_reg = RandomForestRegressor(n_estimators=500, oob_score=True)
rf_reg.fit(X, y)
rf_reg.oob_score_

In [ ]:
et_reg = ExtraTreesRegressor(n_estimators=500, oob_score=True, bootstrap=True)
et_reg.fit(X, y)
et_reg.oob_score_